In [1]:
!pip install transformers langchain pypdf -U langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.6/983.6 kB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.8/355.8 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 11.0 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.2
    Uninstalling transformers-4.41.2:
      Successfully uninstalled transformers-4.41.2


In [2]:
import torch
import nltk.data
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.chains.summarize import load_summarize_chain
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import pipeline

In [6]:
model_name = "MBZUAI/LaMini-Flan-T5-248M"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.50k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

In [7]:
pipe_summarization = pipeline(
        'summarization',
        model = model,
        tokenizer = tokenizer,
        max_length = 200,
        min_length = 25)

In [16]:
def file_preprocessing(file):
    Loader =  PyPDFLoader(file)
    Pages = Loader.load_and_split()
    Text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=200)
    Texts = Text_splitter.split_documents(Pages)
    Processed_Texts = []
    for text in Texts:
        Processed_Texts.append(text.page_content)
    return Processed_Texts

In [11]:
def llm_pipeline(filepath):
    input_text = file_preprocessing(filepath)
    summary = ''
    for text in input_text:
      result = pipe_summarization(text)
      result = result[0]['summary_text']
      summary += result
    return summary

In [20]:
def process_text_in_sentence_chunks(text, max_chunk_size):
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    sentences = tokenizer.tokenize(text)
    current_chunk = ""
    list_text = []
    for sentence in sentences:
        if len(current_chunk) + len(sentence) <= max_chunk_size:
            current_chunk += " " + sentence
        else:
              list_text.append(current_chunk.strip())
              current_chunk = sentence

    if current_chunk:
        list_text.append(current_chunk.strip())
    return list_text

In [17]:
summary_1 = llm_pipeline("/content/Rule Book IITISoC'24.pdf")

Your max_length is set to 200, but your input_length is only 177. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=88)
Your max_length is set to 200, but your input_length is only 195. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=97)


In [18]:
print(len(summary_1),summary_1)

2652 Rules Booklet (IITISoC’24): Participants can register in teams with a minimum size of 2 members and a maximum size of 4. If some participants cannot form a team, the Organizing Committee will allot them a Team. Teams are expected to develop their original work without plagiarism or malpractice. The event comprises three phases: 1. Proposal Sussion: Participants must select up to the top 3 (in order of decreasing priority) from the released Problem Statements and submit a rough proposal on how they aim to solve the problem statement. 2. Project Development: Teams work on their assigned projects under guidance of their allotted mentors. There will be a Mid-Evaluation where the teams are assessed on their progress.The teams must submit their projects in a Github Repository link with a well-documented README file and a 10-15 page presentation describing their project and their contributions. The Organizing Committee will provide certificates of completion only to teams who have succes

Without langchain text splitter

In [23]:
Processed_txt = file_preprocessing("/content/Rule Book IITISoC'24.pdf")
txt = ""
for text in Processed_txt:
  txt += text
Processed_text = process_text_in_sentence_chunks(txt,3000)

In [26]:
summary_2 = ""
for para in Processed_text:
  result_summary = pipe_summarization(para)
  result_summary = result_summary[0]['summary_text']
  summary_2 += result_summary

In [27]:
print(len(summary_2),summary_2)

1617 1. Participants can register in teams with a minimum size of 2 members and a maximum size of 4. If some participants cannot form a team, the Organizing Committee shall allot them a Team. 2. The teams are expected to focus on developing their original work without plagiarism or malpractice. If found, the teams will be immediately disqualified without any prior notice. 3. The event comprises three phases: a. PROPOSAL SUMITION: The participants must select up to the top 3 (in order of decreasing priority) from the released Problem Statements and submit a rough proposal on how they aim to solve the problem statement. (Refer to Appendix A for details) b. PROJECT DEVELOPMENT: The teams work on their assigned projects under guidance of their allotted mentors. There shall be a Mid-Evaluation, where the teams are assessed on their progress. (Note that there are no prerequisiteTeam details: Name, Roll Number, Github ID, LinkedIn profile, and Skills relevant to selected ideas. Project Soluti